In [1]:
import os.path
import numpy as np
import copy

arquivo = os.path.join('projeto.csv') 

dataRDD = (sc.textFile(arquivo, 8))

In [2]:
lpRDD = dataRDD.map(lambda l: l.split(';')).map(lambda l: {'label':float(l[0]),'features':[float(l[1]),float(l[2]),l[3]]})

In [122]:
def contador(lp, feats, splits):
    r = []
    for feat in feats:
        splitss = splits[feat]
        for split in splitss:
            if feat==3:
                r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
            elif feat==0 or feat==1:
                if lp['features'][feat] >= split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
            elif feat==2:                
                if lp['features'][feat] in split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
    return r

In [123]:
def soma(t1,t2):
    return  ((t1[0][0]+t2[0][0],t1[0][1]+t2[0][1]), #((0.668333333, 0), (0.44666944399888886, 0), (1, 0))
             (t1[1][0]+t2[1][0],t1[1][1]+t2[1][1]),
             (t1[2][0]+t2[2][0],t1[2][1]+t2[2][1]))

In [121]:
def calc_stats(t):
    if t[1][2][0] > 0:
        mmenor = t[1][0][0] / t[1][2][0]
        sdmenor = (t[1][1][0] - (t[1][0][0]**2 / t[1][2][0]))/(t[1][2][0])
        cvmenor = np.sqrt(sdmenor)/mmenor*100
        menor = (sdmenor, cvmenor,t[1][2][0])
    else:
        menor = (0, 0, 0)
    
    if t[1][2][1] > 0:    
        mmaior = t[1][0][1] / t[1][2][1]
        sdmaior = (t[1][1][1] - (t[1][0][1]**2 / t[1][2][1]))/(t[1][2][1])
        cvmaior = np.sqrt(sdmaior)/mmaior*100
        maior = (sdmaior, cvmaior,t[1][2][1])
    else:
        maior = (0, 0, 0)
    return (t[0], menor, maior)

In [102]:
statistics = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]<3)
statistics3 = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]==3)

In [103]:
print (statistics.take(1))

[((0, 14.112984), (2.248275898688581, 113.30854423191843, 808782), (1.9512615598751584, 110.24475935122015, 239793))]


In [84]:
classe = statistics3.collect()[0]
print (classe)

((3, 'full'), (0, 0, 0), (1.4767908671038326, 112.69345718620573, 1048575))


In [120]:
def sdreduction(t,c):
    #return ((t[0], c[2][0] - t[1][0]*(t[1][2]/c[2][2]) - t[2][0]*(t[2][2]/c[2][2]))) 
    return ((t[0], c[2][0] - (t[1][2]/c[2][2])*t[1][0] - (t[2][2]/c[2][2])*t[2][0], (t[1][1], t[2][1]))) 

In [105]:
as01 = statistics.map(lambda x: sdreduction(x,classe))

In [111]:
as01.max(key = lambda t: t[1])

((1, 10.633111112), -1.0152223352606939, 58.192296929631738)

In [187]:
def prepara(t):
    return ((t['features'][0],t['features'][0]),(t['features'][1],t['features'][1]))
    
def minmax(t1,t2):
    return ((min(t1[0][0],t2[0][0]),max(t1[0][1],t2[0][1])),
            (min(t1[1][0],t2[1][0]),max(t1[1][1],t2[1][1])))

In [286]:
import time
def tree(lpRDD, maxDepth=1, currDepth=0, caminho=[], st=None,direction='root',verbose=1):
    if st!=None:
        if st[0]<10 or maxDepth<currDepth or st[1]<0:
            media = lpRDD.map(lambda x: x['label']).mean()
            if verbose >= 2:
                print('retorno: caminho: %s média: %s'%(caminho,media))
            return [caminho, (direction, media)]    
    
    t0 = time.time()
    if verbose >= 3:
        print ('total: %s'%lpRDD.count())
    #min0 = lpRDD.map(lambda l: l['features'][0]).min()
    #print ('calc min att 0: %s s'%(time.time()-t0))
    #max0 = lpRDD.map(lambda l: l['features'][0]).max()
    #print ('calc max att 0: %s s'% (time.time()-t0))
    #min1 = lpRDD.map(lambda l: l['features'][1]).min()
    #print ('calc min att 1: %s s'% (time.time()-t0))
    #max1 = lpRDD.map(lambda l: l['features'][1]).max()
    (min0,max0),(min1,max1) =lpRDD.map(prepara).reduce(minmax)
    if verbose >= 1:
        print ( "minmax:", (min0,max0),(min1,max1))
    if verbose >= 2:
        print ('calc minmax: %.2f s'%(time.time()-t0))
    periods = lpRDD.map(lambda l: l['features'][2]).distinct().collect()
    if verbose >= 2:
        print ('calc distinct: %.2f s'%(time.time()-t0))
    #lines = lpRDD.map(lambda l: l['features'][3]).distinct().collect()
    distsplits = [i*(max0-min0)/5. for i in range(1,5)]
    timesplits = [i*(max1-min1)/5. for i in range(1,5)]    
    perisplits = [tuple([periods[j] for j in range(i+1)]) for i in range(len(periods)-1)]

    feats = [0,1,2,3]
    splits = [distsplits,timesplits,perisplits,['full']] 
    results = lpRDD.flatMap(lambda lp: contador(lp,feats,splits)).reduceByKey(soma)
    if verbose >= 2:
        print ('contador: %.2f s'%(time.time()-t0))
    statistics = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]<3)
    if verbose >= 2:
        print ('estatísticas: %.2f s'%(time.time()-t0))
    statistics3 = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]==3)
    classe = statistics3.collect()[0]
    es = statistics.map(lambda x: sdreduction(x,classe)).collect()  
    e = max(es, key = lambda t: t[1]) #atributo / split escolhido
    if verbose >= 2:
        print ('atributo escolhido: %.2f s'%(time.time()-t0))
    if e[0][0]<2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]<e[0][1])
            if verbose >= 2:
                print ('filtro menor01: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]>e[0][1])   
            if verbose >= 2:
                print ('filtro maior01: %.2f s'%(time.time()-t0))
    elif e[0][0]==2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] in e[0][1])
            if verbose >= 2:
                print ('filtro menor2: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] not in e[0][1])
            if verbose >= 2:
                print ('filtro maior2: %.2f s'%(time.time()-t0))
    
    caminho.append((direction,e[0]))
    camenor = copy.deepcopy(caminho)
    camaior = copy.deepcopy(caminho)
    

    amenor = tree(menorRDD, maxDepth, currDepth+1,camenor, (e[2][0], e[1]),'menor',verbose=2)
  

    amaior = tree(maiorRDD, maxDepth, currDepth+1,camaior, (e[2][1], e[1]),'maior',verbose=2)
    return [amenor, amaior]

In [288]:
ti=time.time()
arvore = tree(lpRDD,maxDepth=2,verbose=2)
tf = time.time()

minmax: (0.0003, 17.64153) (0.0, 13.29138889)
calc minmax: 12.12 s
calc distinct: 27.54 s
contador: 27.56 s
estatísticas: 27.56 s
atributo escolhido: 87.90 s
filtro menor01: 87.90 s
filtro maior01: 87.90 s
minmax: (0.0003, 17.64153) (0.0, 2.658055556)
calc minmax: 19.52 s
calc distinct: 35.22 s
contador: 35.24 s
estatísticas: 35.24 s
atributo escolhido: 95.72 s
filtro menor01: 95.73 s
filtro maior01: 95.73 s
minmax: (0.0003, 17.64153) (0.0, 1.594722222)
calc minmax: 17.79 s
calc distinct: 42.11 s
contador: 42.13 s
estatísticas: 42.13 s
atributo escolhido: 106.40 s
filtro menor01: 106.40 s
filtro maior01: 106.40 s
retorno: caminho: [('root', (1, 2.658277778)), ('root', (1, 2.658277778)), ('menor', (1, 1.5948333336)), ('menor', (1, 1.2757777775999999))] média: 1.0777383456334122
retorno: caminho: [('root', (1, 2.658277778)), ('root', (1, 2.658277778)), ('menor', (1, 1.5948333336)), ('menor', (1, 1.2757777775999999))] média: 2.54314692416784
minmax: (0.05143, 17.62744) (1.595, 2.658055556

#Resultado com maxDepth=4

total: 1048575
minmax: (0.0003, 17.64153) (0.0, 13.29138889)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((1, 2.658277778), 1.128366881380903, (84.378606643153233, 29.309996406197286)) root
total: 1024936
minmax: (0.0003, 17.64153) (0.0, 2.658055556)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((1, 1.5948333336), 0.16114889541672692, (74.161213828278576, 64.976608489544191)) menor
total: 1011609
minmax: (0.0003, 17.64153) (0.0, 1.594722222)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((1, 1.2757777775999999), 0.036454756507950714, (69.090526924911728, 88.677292985413828)) menor
total: 994134
minmax: (0.0003, 17.64153) (0.0, 1.275555556)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((1, 1.0204444448), 0.013481143219890787, (65.191035692835456, 86.456220549612581)) menor
total: 942267
minmax: (0.0003, 17.64069) (0.0, 1.020277778)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((2, ('m',)), 0.004144635885719998, (67.671517899127153, 38.67562143709295)) menor
total: 51867
minmax: (3.73532, 17.64153) (1.020555556, 1.275555556)
[('m',), ('m', 'pm'), ('m', 'pm', 'pt'), ('m', 'pm', 'pt', 'n')]
((2, ('m', 'pm')), 0.03582006682450771, (102.80546816158792, 26.67224971372044)) maior
total: 17475
minmax: (0.32818, 17.64088) (1.275833333, 1.594722222)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((0, 10.38762), 0.4791027147972322, (77.29525744579378, 84.451800696190432)) maior
total: 3331
minmax: (0.32818, 10.38666) (1.275833333, 1.594444444)
[('m',), ('m', 'pt'), ('m', 'pt', 'n'), ('m', 'pt', 'n', 'pm')]
((2, ('m', 'pt', 'n', 'pm')), 0.6825293842868456, (74.036182652296674, 72.53442702043121)) menor
total: 14144
minmax: (10.39463, 17.64088) (1.275833333, 1.594722222)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((2, ('m', 'n', 'pt')), 0.3394539907347749, (48.072663011928249, 86.669252995347534)) maior
total: 13327
minmax: (0.05143, 17.62744) (1.595, 2.658055556)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((0, 14.060808), 1.5492580159304148, (54.142191929479274, 77.65940377504225)) maior
total: 9097
minmax: (0.05143, 14.05513) (1.595, 2.658055556)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((0, 11.202960000000001), 0.47293097455352684, (63.823123677220785, 36.260553462264625)) menor
total: 5790
minmax: (0.05143, 11.20233) (1.595, 2.656944444)
[('m',), ('m', 'pt'), ('m', 'pt', 'n')]
((2, ('m', 'pt')), 0.2937912769802935, (69.849463692676466, 59.018424294024818)) menor
total: 3307
minmax: (11.20352, 14.05513) (1.595, 2.658055556)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((2, ('m', 'n', 'pt')), 0.15142227020266574, (84.066198056855001, 32.6851660164726)) maior
total: 4230
minmax: (14.06088, 17.62744) (1.595, 2.656944444)
[('m',), ('m', 'n'), ('m', 'n', 'pt'), ('m', 'n', 'pt', 'pm')]
((2, ('m', 'n', 'pt')), 0.5552252766658738, (63.107897616868428, 73.320454834047524)) maior
total: 1986
minmax: (14.06088, 17.59387) (1.595, 2.656944444)
[('m',), ('m', 'n')]
((2, ('m',)), 0.8182346254663307, (68.980414776351381, 27.80748764419253)) menor
total: 2244
minmax: (14.06164, 17.62744) (1.595, 2.654166667)
[('pm',)]
((2, ('pm',)), 0.7623849343736041, (64.924364248796593, 4.2205084451627739)) maior
total: 23639
minmax: (0.13708, 17.63388) (2.658333333, 13.29138889)
[('m',), ('m', 'n'), ('m', 'n', 'pt')]
((2, ('m', 'n')), 1.4161762689132464, (26.037399941247163, 23.9750483308467)) maior
total: 11881
minmax: (0.38383, 17.58191) (2.658333333, 13.29138889)
[('m',)]
((1, 8.5064444456), 0.6365935736943849, (21.628780502997579, 7.6032978499937345)) menor
total: 11671
minmax: (0.38383, 17.58191) (2.658333333, 8.500277778)
[('m',)]
((1, 4.673555555999999), 0.1934913791657552, (24.137300723141507, 17.516320197253705)) menor
total: 5806
minmax: (7.19508, 17.58191) (2.658333333, 4.673333333)
[('m',)]
((0, 8.309464), 0.02044882967282735, (0.89210420666222834, 24.038023095573017)) menor
total: 5865
minmax: (0.38383, 17.55145) (4.673611111, 8.500277778)
[('m',)]
((2, ('m',)), 0.604488449743678, (12.824725443066662, 24.574239255064828)) maior
total: 11758
minmax: (0.13708, 17.63388) (2.658333333, 12.80333333)
[('pt',)]
((1, 4.0579999988), 1.284640924710557, (43.1443383351669, 15.257521661273834)) maior
total: 3303
minmax: (0.13708, 17.63388) (2.658333333, 4.0575)
[('pt',)]
((0, 13.997440000000001), 0.5184265883332406, (41.010142574946855, 44.944991463798878)) menor
total: 2284
minmax: (0.13708, 13.98282) (2.658333333, 4.0575)
[('pt',)]
((0, 11.076592000000002), 2.0756899499911152, (49.256899942318981, 28.296676917747387)) menor
total: 1019
minmax: (14.00687, 17.63388) (2.658611111, 4.056388889)
[('pt',)]
((2, ('pt',)), 0.028383069323637322, (42.162647128901462, 45.530067230810126)) maior
total: 8455
minmax: (3.06314, 17.53385) (4.059444444, 12.80333333)
[('pt',)]
((2, ('pt',)), 0.33387912957540655, (21.124430198369996, 10.592577246457033)) maior
total: 6637
minmax: (3.06314, 17.30937) (4.059444444, 11.36805556)
[]
((1, 5.8468888928), 0.037546087032547915, (13.873763918561041, 8.5148943215227657)) menor
total: 1818
minmax: (3.69633, 17.53385) (4.063333333, 12.80333333)
[]
((0, 11.070016), 1.4712799538064196, (24.8271855081118, 18.02098341859385)) maior

In [274]:
print('Tempo total de construção: %.2f'%(tf-ti))

Tempo total de construção: 3118.13


In [273]:
arvore

[[[[[[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('menor', (2, ('m',)))],
      ('menor', 0.9068417788864728)],
     [[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('menor', (2, ('m',)))],
      ('maior', 1.0793486041087266)]],
    [[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('maior', (2, ('m', 'pm')))],
      ('menor', 1.3659583908570467)],
     [[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('maior', (2, ('m', 'pm')))],
      ('maior', 1.7459477766995652)]]],
   [[[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757

In [275]:
min0 = lpRDD.map(lambda l: l['label']).min()
max0 = lpRDD.map(lambda l: l['label']).max()